PRAC01 Tipologia UOC Curso 2022

Scrapper de Información de Masteres

    WEB: http://www.emagister.com

Equipo: 

    Angel A. Urbina 
    Almudena Caballero

Fecha:

    05 Abril 2022

Versión: 1.3

Notas:
    
    Incluye análisis Secuencial y Multiproceso


In [1]:
# Construcción de Loggings para informar

# Objetivo: Disponer tanto en pantalla como en file (log) la información del procesamiento que se realiza 

import logging
import logging.config

# Referencia:
# https://coderzcolumn.com/tutorials/python/logging-config-simple-guide-to-configure-loggers-from-dictionary-and-config-files-in-python

# Carga de Archivo de configuracón del login
logging.config.fileConfig('loggingUOCPRA01.conf')

# Creación del logger
logger = logging.getLogger('UOCPRA01')

# Tipos de Loggers que se podrian generar:
# 'application' code
#logger.debug('debug message')
#logger.info('info message')
#logger.warning('warn message')
#logger.error('error message')
#logger.critical('critical message')

In [2]:
# Librerias Necesarias

from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import re

import os

import requests
import random
import time
from time import sleep

# Normalización textos (quitar codigos con acentos)
from unicodedata import normalize


2022-04-05 17:13:00,062 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


# 00.- Funciones Auxiliares

In [33]:
# Extraccion informacion del 'nombre archivo' de un URL que lo contiene
# Función Auxiliar

# Basado en:
# https://stackoverflow.com/questions/66876071/extracting-a-complex-substring-using-regex-with-data-from-a-string-in-python
# https://stackoverflow.com/questions/14473180/regex-to-get-a-filename-from-a-url

def ExtraeName(cadena):
    """
    Función extraccion informacion del nombre archivo de un URL
    ExtraeName(cadena)
    entrada:
        cadena --> Url
    return:
        string final del url
    """
    
    regex=r"(?<=\/)[^\/\?#]+(?=[^\/]*$)"
    matches = re.findall(regex, cadena)
    return str(matches)


In [34]:
# Generación Directorio y Devuelve Path + Archivo en el Directorio construido
# Funcion Auxiliar 

def GenDirecFile(path, filename):
    """
    GenDirecFile(path, filename, r)
    Entrada:
        path -> Path del directorio a generar
        filename -> Nombre Archivo a guardar
    Devuelve:
        path y filename
    """
    # Generación directorio si no existe
    if not os.path.exists(path):
        os.makedirs(path)
    # Grabación en el directorio en formato binario (imagenes)
    return os.path.join(path, filename)

In [35]:
# Descarga Imagenes
# Funcion Auxiliar 

def DescargaImagenes(path, link, institucion):
    """
    DescargaImagenes(path, link, institucion)
    Entrada:
        path -> Path del directorio a generar
        link -> Link a el archivo a descargar
        institucion -> Nombre de la institucion cuyo logo es la imagen
    Devuelve:
        Archivo de Imagenes descargado en el directorio path
    """
    # Delays para evitar problemas
    time.sleep(random.randint(1,2))
    # Obtener Imagen
    #r = requests.get(s, allow_redirects=False)   
    r = requests.get(link, allow_redirects=True)
    # Generar Nombre Archivo
    nombre = str(institucion)
    # Directorio donde se generara salida
    file01=GenDirecFile(str(path), nombre)
    # Grabacion resultados
    with open(file01, 'wb') as file:
        file.write(r.content)

In [36]:
# Grabación Archivo en Directorio
# Funcion Auxiliar 

# path = IMAGENES
# filename = img_alt + '.jpg'
# buffer = r.content
# open(nombre, 'wb').write(r.content)

def GraFileDirec(path, filename, r):
    """
    GraFileDirec(path, filename, r)
    path -> Path del directorio a generar
    filename -> Nombre Archivo a guardar
    r --> Objeto BeatifulSoup
        r.content -> Contenido Stream del Objeto
    """
    # Generación directorio si no existe
    if not os.path.exists(path):
        os.makedirs(path)
    # Grabación en el directorio en formato binario (imagenes)
    with open(os.path.join(path, filename), 'wb') as temp_file:
        temp_file.write(r.content)

# 01.- Análisis de Links asociados a la WEB

In [6]:
# 01.01- El primer paso es la identificación de los links asociados a cada Master

# Pagina WEB a analizar

URLWebPageAnalizar = 'https://www.emagister.com/master/web/sitemap'

logger.info('Analisis WEB %s', URLWebPageAnalizar)

# Lista donde se almacenaran los links encontrados
ListaWebs=[]

# Abrir Web
html = urlopen(URLWebPageAnalizar)

# Generar objeto BeatifulSoup
bs = BeautifulSoup(html, 'html.parser')
for link in bs.find_all('a'):
    # Busca todos los elementos que esten en 'href'
    try:
        StringWeb = link.attrs['href']
    except:
        continue
    # Comprueba que lo obtenido se corresponda con el formato de una pagina WEB
    # match (True si es un link / False en resto)
    match = re.search(r'https?://[^\s<>"]+|www\.[^\s<>"]+', str(StringWeb)) 
    if match:      
        ListaWebs.append(link.attrs['href'])
        logger.info('Encontrado link %s', link.attrs['href'])

# Guardar links obtenidos en Archivo Auxiliar FILE_Links
# - Nombre Archivo
FILE_Links = '01_ListaWebs.xlsx'
# - Genera DataFrame apartir de la lista de WEBs generadas
df = pd.DataFrame(ListaWebs)
# - Escribe Archivo
df.to_excel(FILE_Links)
logger.info('Generado Archivo %s', FILE_Links)

2022-04-05 17:18:33,701 - UOCPRA01 - INFO - Analisis WEB https://www.emagister.com/master/web/sitemap
2022-04-05 17:18:34,133 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/web/pagina/legal#sixth
2022-04-05 17:18:34,134 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/
2022-04-05 17:18:34,134 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/
2022-04-05 17:18:34,134 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/
2022-04-05 17:18:34,136 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/universidades/
2022-04-05 17:18:34,136 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/
2022-04-05 17:18:34,137 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/formacion-profesional/
2022-04-05 17:18:34,137 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/oposiciones-convocatorias/
2022-04-05 17:18:34,138 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/subvenciona

2022-04-05 17:18:34,166 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-pedagogia-tematica-479.htm
2022-04-05 17:18:34,166 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-preparacion-examenes-tematica-826.htm
2022-04-05 17:18:34,167 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-cultura-humanidades-tematica-11.htm
2022-04-05 17:18:34,167 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-antropologia-tematica-745.htm
2022-04-05 17:18:34,167 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-artes-escenicas-tematica-747.htm
2022-04-05 17:18:34,168 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-ciencias-informacion-tematica-751.htm
2022-04-05 17:18:34,168 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-ciencias-sociales-tematica-752.htm
2022-04-05 17:18:34,168 - UOCPRA01 - INFO - Encontrado link htt

2022-04-05 17:18:34,195 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-investigacion-cientifica-tematica-476.htm
2022-04-05 17:18:34,195 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-matematicas-tematica-472.htm
2022-04-05 17:18:34,196 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-quimica-tematica-32.htm
2022-04-05 17:18:34,196 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-inmobiliaria-construccion-tematica-528.htm
2022-04-05 17:18:34,197 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-arquitectura-tematica-526.htm
2022-04-05 17:18:34,197 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-construccion-tematica-758.htm
2022-04-05 17:18:34,198 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-gestion-inmobiliaria-tematica-179.htm
2022-04-05 17:18:34,198 - UOCPRA01 - INFO - Encontrado link http

2022-04-05 17:18:34,225 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/centros.htm
2022-04-05 17:18:34,225 - UOCPRA01 - INFO - Encontrado link https://formacio.lavanguardia.com/
2022-04-05 17:18:34,226 - UOCPRA01 - INFO - Encontrado link https://www.emagister.fr/
2022-04-05 17:18:34,227 - UOCPRA01 - INFO - Encontrado link https://www.emagister.co.uk/
2022-04-05 17:18:34,227 - UOCPRA01 - INFO - Encontrado link https://www.emagister.it/
2022-04-05 17:18:34,228 - UOCPRA01 - INFO - Encontrado link https://www.emagister.de/
2022-04-05 17:18:34,228 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com.mx/
2022-04-05 17:18:34,229 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com.co/
2022-04-05 17:18:34,229 - UOCPRA01 - INFO - Encontrado link https://www.emagister.cl/
2022-04-05 17:18:34,230 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com.ar/
2022-04-05 17:18:34,230 - UOCPRA01 - INFO - Encontrado link https://www.emagister.in/
2022-04-05 17:18:3

In [7]:
# Nos informa del Número Total de Links encontrados
logger.info('Encontrado %s links', len(ListaWebs))

2022-04-05 17:18:45,610 - UOCPRA01 - INFO - Encontrado 206 links


In [8]:
# 01.02.- Función para Construcción del Diccionario de Información de los links encontrados.

diccionario={}

def RecorrerWebs(ListaWebs):
    """
    Construye Diccionario Valores de una lista de Webs
    
    Funcion:
       RecorrerWebs( ListaWeb)
    Entrada:
        ListaWeb --> Lista de Direcciones de Web
    Salida:
        Diccionario con información:
            Keys: 
                Indice correlativo
            Values:
                - Pagina donde buscamos
                - Titulo Master
                - Link al master concreto
    """
    
    # Leer Webs obtenidas paso previo
    # - indice
    indice = 0
    # - Delay 1 seg para no ser banneado
    sleep(1)
    
    for indi, ListaWeb in enumerate(ListaWebs):
        # Delay 1 seg para no ser banneado
        sleep(1)
        # Captura de posibles errores Webs
        try:
            html = urlopen(ListaWeb)
        except HTTPError as e:
            # Lanza un aviso con indice de ListaWebs y error
            logger.warning('%s warning %s',indi, e)
            continue
        except URLError as e:
            # Lanza aviso si no encuentra la Web
            logger.warning('%s ',ListaWeb)
            logger.warning('%s ',e)               
            continue
        # Captura posibles errores Beatifulsoup
        try:
            bs = BeautifulSoup(html, 'html.parser')
        except AttributeError as e:
            logger.warning('%s warning %s',indi, e)              
            continue
        
        # Captura de información básica
        divs = bs.find_all(class_ ="course-box-item title-box")
    
        for s in divs:
            inks = s.find_all('h3')
            for t in inks:
                lista = t.find_all('a')
                # Obtención indice y valor de lista
                #for indice, x in enumerate(lista):
                for x in lista:
                    indice = indice+1
                    # Titulo 
                    x2 = x.attrs['title']
                    # Referencia al titulo
                    x1 = x.attrs['href']
                    logger.info('%s: %s',indice, x2)

                    # Construccion diccionario salida
                    diccionario[indice] = [ListaWeb,x2,x1]
    return diccionario

In [9]:
# 01.03.- Construcción Efectiva del Diccionario

# Nombre Diccionario Construido:
#    a --> diccionario

a = RecorrerWebs(ListaWebs)

logger.info('Construido Diccionario')

2022-04-05 17:19:31,164 - UOCPRA01 - INFO - 1: Máster MBA en dirección de centros hidrotermales
2022-04-05 17:19:31,164 - UOCPRA01 - INFO - 2: Máster en Nutrición y Salud
2022-04-05 17:19:31,165 - UOCPRA01 - INFO - 3: Máster en Biotecnología de la Salud - Madrid
2022-04-05 17:19:31,166 - UOCPRA01 - INFO - 4: Máster Universitario en Nutrición y Salud
2022-04-05 17:19:31,166 - UOCPRA01 - INFO - 5: Máster Profesional en Gestión Sanitaria
2022-04-05 17:19:31,167 - UOCPRA01 - INFO - 6: MÁSTER EN SALUD MENTAL
2022-04-05 17:19:31,168 - UOCPRA01 - INFO - 7: Máster en trabajo social en servicios de la salud y Máster en salud mental
2022-04-05 17:19:31,168 - UOCPRA01 - INFO - 8: Máster en Nutrición y Salud - A distancia
2022-04-05 17:19:31,169 - UOCPRA01 - INFO - 9: Máster en Salud mental en el anciano y Máster en la Enfermedad del Alzheimer ( Con prácticas)
2022-04-05 17:19:31,169 - UOCPRA01 - INFO - 10: Máster en Nutrición y Salud & Coaching, Gestión Emocional y Mindfulness - A distancia
2022-

2022-04-05 17:19:36,212 - UOCPRA01 - INFO - 75: MÁSTER EN DIRECCIÓN DE CLÍNICAS DE FISIOTERAPIA
2022-04-05 17:19:36,212 - UOCPRA01 - INFO - 76: MÁSTER EN TRATAMIENTO DE LESIONES DEPORTIVAS PARA FISIOTERAPEUTAS - CON CERTIFICACIÓN UNIVERSITARIA INTERNACIONAL Y RECONOCIMIENTO DE 30 ECTS
2022-04-05 17:19:36,213 - UOCPRA01 - INFO - 77: POSTGRADO EN FISIOTERAPIA Y QUIROMASAJE AVANZADOS - ONLINE
2022-04-05 17:19:36,214 - UOCPRA01 - INFO - 78: Máster en tratamiento de lesiones deportivas
2022-04-05 17:19:36,215 - UOCPRA01 - INFO - 79: POSTGRADO EN REHABILITACIÓN ACUÁTICA
2022-04-05 17:19:36,215 - UOCPRA01 - INFO - 80: Máster de formación permanente en readaptación deportiva
2022-04-05 17:19:36,216 - UOCPRA01 - INFO - 81: Máster Universitario en Terapia Manual Ortopédica en el Tratamiento del Dolor
2022-04-05 17:19:36,217 - UOCPRA01 - INFO - 82: MÁSTER EN DIRECCIÓN DE CLÍNICAS DE FISIOTERAPIA
2022-04-05 17:19:36,217 - UOCPRA01 - INFO - 83: Máster en rehabilitación deportiva
2022-04-05 17:19:37

2022-04-05 17:19:43,046 - UOCPRA01 - INFO - 153: Máster en terapias manuales y complementarias - Bilbao
2022-04-05 17:19:43,046 - UOCPRA01 - INFO - 154: Máster de experto en radiestesia, geobiología y feng shui
2022-04-05 17:19:43,047 - UOCPRA01 - INFO - 155: Máster en herbodietética
2022-04-05 17:19:43,048 - UOCPRA01 - INFO - 156: Máster en homeopatía, fitoterapia y nutrición
2022-04-05 17:19:43,048 - UOCPRA01 - INFO - 157: MÁSTER EN ACUPUNTURA - ONLINE
2022-04-05 17:19:44,671 - UOCPRA01 - INFO - 158: Máster en Nutrición Vegetariana + Máster Experto en Coaching Nutricional
2022-04-05 17:19:44,672 - UOCPRA01 - INFO - 159: Máster en Nutrición y Salud
2022-04-05 17:19:44,672 - UOCPRA01 - INFO - 160: Máster Universitario en Nutrición y Salud
2022-04-05 17:19:44,673 - UOCPRA01 - INFO - 161: Técnico experto en dietética y nutrición más máster experto en coach nutricional
2022-04-05 17:19:44,674 - UOCPRA01 - INFO - 162: Master en Nutrición Deportiva
2022-04-05 17:19:44,675 - UOCPRA01 - INFO 

2022-04-05 17:19:49,646 - UOCPRA01 - INFO - 231: Experto universitario en ortopedia
2022-04-05 17:19:49,647 - UOCPRA01 - INFO - 232: Experto Universitario en Podología Basada en la Práctica Clínica
2022-04-05 17:19:49,647 - UOCPRA01 - INFO - 233: Experto en Podología Basada en la Práctica Clínica
2022-04-05 17:19:49,648 - UOCPRA01 - INFO - 234: Master en Podología Clínica Avanzada
2022-04-05 17:19:49,648 - UOCPRA01 - INFO - 235: Experto Universitario en Cirugía Podológica
2022-04-05 17:19:49,649 - UOCPRA01 - INFO - 236: Experto en Cirugía Podológica
2022-04-05 17:19:51,508 - UOCPRA01 - INFO - 237: Máster en psicología infantil y adolescente más máster en coaching y en inteligencia emocional infantil y juvenil
2022-04-05 17:19:51,509 - UOCPRA01 - INFO - 238: Máster en psicología infantil y adolescente más máster en coaching y en inteligencia emocional infantil y juvenil
2022-04-05 17:19:51,509 - UOCPRA01 - INFO - 239: Máster en Psicología General
2022-04-05 17:19:51,510 - UOCPRA01 - INF

2022-04-05 17:19:58,139 - UOCPRA01 - INFO - 305: Máster en análisis clínicos en laboratorio para veterinaria
2022-04-05 17:19:58,139 - UOCPRA01 - INFO - 306: MÁSTER EN URGENCIAS VETERINARIAS Y QUIRÓFANO
2022-04-05 17:19:58,147 - UOCPRA01 - INFO - 307: Máster en urgencias veterinarias y quirófano
2022-04-05 17:19:58,148 - UOCPRA01 - INFO - 308: TÉCNICO SUPERIOR VETERINARIO EN ASISTENCIA Y RECUPERACIÓN DE ESPECIES SALVAJES
2022-04-05 17:19:58,148 - UOCPRA01 - INFO - 309: Master en Farmacia Veterinaria + Titulación Universitaria
2022-04-05 17:19:58,149 - UOCPRA01 - INFO - 310: Máster en asistencia a la atención clínica en centros veterinarios
2022-04-05 17:19:58,149 - UOCPRA01 - INFO - 311: Máster en análisis clínicos en laboratorio para veterinaria
2022-04-05 17:19:58,150 - UOCPRA01 - INFO - 312: Máster en quirófano veterinario
2022-04-05 17:19:58,150 - UOCPRA01 - INFO - 313: MÁSTER EN DIETÉTICA Y NUTRICIÓN VETERINARIA - CON CERTIFICACIÓN UNIVERSITARIA INTERNACIONAL Y RECONOCIMIENTO DE 3

...

2022-04-05 17:24:20,880 - UOCPRA01 - INFO - 2694: MÁSTER EN GESTIÓN DE CALIDAD + AUDITOR DE GESTIÓN DE CALIDAD (ISO 9001/2015 - ISO 19011) - DIPLOMA AUTENTIFICADO POR NOTARIO EUROPEO -
2022-04-05 17:24:20,880 - UOCPRA01 - INFO - 2695: MÁSTER EN GERENCIA DE CLUBES DEPORTIVOS + MÁSTER EN SPORTS MANAGEMENT
2022-04-05 17:24:20,881 - UOCPRA01 - INFO - 2696: MÁSTER EXPERTO EN CARTOGRAFÍA
2022-04-05 17:24:20,882 - UOCPRA01 - INFO - 2697: MÁSTER EN SOCIAL MEDIA MANAGEMENT - DIPLOMA AUTENTIFICADO POR NOTARIO EUROPEO -
2022-04-05 17:24:20,883 - UOCPRA01 - INFO - 2698: POSTGRADO EN TRAZADOS DE CARRETERAS Y VÍAS URBANAS
2022-04-05 17:24:20,884 - UOCPRA01 - INFO - 2699: MÁSTER EN TÉCNICAS Y COSMÉTICA HIDROTERMAL - CERTIFICADO POR APENB - ASOCIACIÓN PROFESIONAL ESPAÑOLA DE NATUROPATÍA Y BIOTERAPIA -
2022-04-05 17:24:20,885 - UOCPRA01 - INFO - 2700: MÁSTER EN PREVENCIÓN DE RIESGOS LABORALES (ERGONOMÍA Y RIESGOS PSICOSOCIALES) - DIPLOMA AUTENTIFICADO POR NOTARIO EUROPEO -
2022-04-05 17:24:20,886 - UOC

2022-04-05 17:24:33,497 - UOCPRA01 - INFO - 2767: Master en Transporte y Gestión Logística
2022-04-05 17:24:33,498 - UOCPRA01 - INFO - 2768: Máster En Asesoramiento Empresarial + Curso De Consultor En Logística y Transporte - con posibilidad de prácticas-
2022-04-05 17:24:33,499 - UOCPRA01 - INFO - 2769: Master of Engineering in Logistics & Supply Chain Management (ZLOG)
2022-04-05 17:24:33,499 - UOCPRA01 - INFO - 2770: Máster en Gestión y Dirección Logística
2022-04-05 17:24:33,500 - UOCPRA01 - INFO - 2771: Máster en Dirección Estratégica de Operaciones e Innovación
2022-04-05 17:24:33,501 - UOCPRA01 - INFO - 2772: Máster Universitario en Dirección Comercial y Ventas
2022-04-05 17:24:33,501 - UOCPRA01 - INFO - 2773: Master en Transporte y Gestión Logística + Curso Habilidades y Destrezas para la Gestión de Equipos (3C)
2022-04-05 17:24:33,502 - UOCPRA01 - INFO - 2774: Máster de Excelencia en Química
2022-04-05 17:24:36,030 - UOCPRA01 - INFO - 2775: Máster en Gestión y Dirección Logíst

2022-04-05 17:24:50,143 - UOCPRA01 - INFO - 2848: Máster Oficial - Didáctica del Chino para Hispanohablantes
2022-04-05 17:24:50,144 - UOCPRA01 - INFO - 2849: Máster Oficial - Unión Europea-China: Cultura y Economía
2022-04-05 17:24:50,144 - UOCPRA01 - INFO - 2850: Estudios Chinos
2022-04-05 17:24:50,145 - UOCPRA01 - INFO - 2851: Estudis Xinesos
2022-04-05 17:24:50,145 - UOCPRA01 - INFO - 2852: Curso de Chino Mandarín Nivel Oficial A1 A2 online
2022-04-05 17:24:50,146 - UOCPRA01 - INFO - 2853: Curso Online de Chino Práctico
2022-04-05 17:24:50,146 - UOCPRA01 - INFO - 2854: Chino para Niños Sábados (6-16 años)
2022-04-05 17:24:50,147 - UOCPRA01 - INFO - 2855: Chino
2022-04-05 17:24:50,148 - UOCPRA01 - INFO - 2856: Chino. Curso Anual Adultos
2022-04-05 17:24:50,148 - UOCPRA01 - INFO - 2857: Pregrado en Lenguas y Cultura
2022-04-05 17:24:50,149 - UOCPRA01 - INFO - 2858: Chino Nivel Básico - EOI
2022-04-05 17:24:50,149 - UOCPRA01 - INFO - 2859: Chino 1
2022-04-05 17:24:50,150 - UOCPRA01 - 

2022-04-05 17:25:09,057 - UOCPRA01 - INFO - 2931: Curso Intensivo de Italiano básico - ONLINE
2022-04-05 17:25:09,058 - UOCPRA01 - INFO - 2932: ITALIANO BASICO DESDE CERO A1
2022-04-05 17:25:09,058 - UOCPRA01 - INFO - 2933: Curso online de italiano nivel a1
2022-04-05 17:25:09,059 - UOCPRA01 - INFO - 2934: Curso online de italiano en 3 niveles con 12 meses de acceso online
2022-04-05 17:25:09,060 - UOCPRA01 - INFO - 2935: ITALIANO B1
2022-04-05 17:25:09,060 - UOCPRA01 - INFO - 2936: Clases particulares de italiano
2022-04-05 17:25:09,061 - UOCPRA01 - INFO - 2937: ITALIANO - BÁSICO - NIVEL IV
2022-04-05 17:25:09,061 - UOCPRA01 - INFO - 2938: Italiano A2
2022-04-05 17:25:09,062 - UOCPRA01 - INFO - 2939: ITALIANO - A1 LIVELLO 1
2022-04-05 17:25:09,063 - UOCPRA01 - INFO - 2940: Italiano Livello C1
2022-04-05 17:25:09,063 - UOCPRA01 - INFO - 2941: Iniciación al italiano
2022-04-05 17:25:09,064 - UOCPRA01 - INFO - 2942: 15 Clases particulares de italiano
2022-04-05 17:25:09,064 - UOCPRA01 - 

2022-04-05 17:25:23,102 - UOCPRA01 - INFO - 3026: Máster Oficial - Tradumática: Tecnologías de la Traducción
2022-04-05 17:25:23,103 - UOCPRA01 - INFO - 3027: Traducción Inglés-español
2022-04-05 17:25:23,104 - UOCPRA01 - INFO - 3028: Traducción e Interculturalidad
2022-04-05 17:25:23,104 - UOCPRA01 - INFO - 3029: Máster de formación permanente en traducción especializada, mención italiano
2022-04-05 17:25:23,105 - UOCPRA01 - INFO - 3030: Máster en Traducción e Interpretación
2022-04-05 17:25:23,106 - UOCPRA01 - INFO - 3031: Máster Universitario en Traducción Especializada
2022-04-05 17:25:23,106 - UOCPRA01 - INFO - 3032: Estudios de Traducción
2022-04-05 17:25:23,107 - UOCPRA01 - INFO - 3033: Máster Oficial - Traducción Audiovisual
2022-04-05 17:25:23,108 - UOCPRA01 - INFO - 3034: Máster Propio en Interpretación de Conferencias
2022-04-05 17:25:25,623 - UOCPRA01 - INFO - 3035: Máster Universitario en Seguridad, Defensa y Geoestrategia
2022-04-05 17:25:25,624 - UOCPRA01 - INFO - 3036: 

01.04.- Guardamos la información obtenida en un Archivo para ello:

        01.04.01.- Generamos Dataframe
        01.04.02.- Generamos File a partir del Dataframe

 Pasos:

In [13]:
# 01.04.01.- Generación de Dataframe a partir del Diccionario

# Referencia:
# https://stackoverflow.com/questions/54416620/dictionary-with-multiple-key-values-to-dataframe

TablaResult = (pd.DataFrame.from_dict(a, orient='index')
                  .rename(columns=lambda x: x+1)
                  .add_prefix('n')
                  .rename_axis('indice')
                  .reset_index())

logger.info('Construida Tabla TablaResult')

# Poner nombres columnas
TablaResult.columns = ['key','Pagina_Origen', 'Titulo', 'Pag_Referencia']

# Poner de indice a key
TablaResult=TablaResult.set_index('key')

# Verificación Construcción Ok( No necesario )
# TablaResult.head(3)

logger.info('Tamaño tabla  %s', TablaResult.shape)
logger.info('Columnas  %s', TablaResult.columns)

2022-04-05 17:26:43,574 - UOCPRA01 - INFO - Construida Tabla TablaResult
2022-04-05 17:26:43,577 - UOCPRA01 - INFO - Tamaño tabla  (3102, 3)
2022-04-05 17:26:43,577 - UOCPRA01 - INFO - Columnas  Index(['Pagina_Origen', 'Titulo', 'Pag_Referencia'], dtype='object')


In [14]:
# 01.04.02.- Generación Archivo parcial de resultados

# Generación Nombres Archivos a generar
from datetime import datetime

nombre=datetime.today().strftime('%d-%m-%y')

FILE_01 = "LINK-"+nombre+".xlsx"

TablaResult.to_excel(FILE_01)
logger.info('Generado Archivo %s', FILE_01)

2022-04-05 17:27:03,967 - UOCPRA01 - INFO - Generado Archivo LINK-05-04-22.xlsx


# 02.- Analisis de Links Identificados 

    02.1- Para ello leeremos Archivo del paso previo
    02.2- Haremos el analisis

In [15]:
# 02.01.- Identificación Archivos en nuestra Maquina

# Directorio donde estamos
cwd=os.getcwd()
print("-----------------------------------------------------------------------")
print("A.- Directorio en maquina donde estamos")
print (cwd)

# Lista de Archivos disponibles en directorio 'Directorio'
Directorio = cwd
print("-----------------------------------------------------------------------")
print("C.- Listado de Archivos Disponibles")
ListaFile=os.listdir(Directorio)

# Selección Archivos que hayan sido generados en lanzamientos previos del pto 01.
# - Uso Regex 
cadena=r"^LINK-"
prog1 = re.compile(cadena)

# Extraer lista de archivos que cumplan criterio

Archivos_FILT=[]
for s in ListaFile:
    if prog1.search(s):
        Archivos_FILT.append(s)
# Listado de Nombres Archivos validos
Archivos_FILT

-----------------------------------------------------------------------
A.- Directorio en maquina donde estamos
/home/anson/Descargas/01_UOC/05_TIPOLOGIA/PRA01
-----------------------------------------------------------------------
C.- Listado de Archivos Disponibles


['LINK-03-04-22.xlsx', 'LINK-05-04-22.xlsx']

In [21]:
# Escogeremos el archivo que nos interese de los identificados en el paso previo

# Leer Archivo
FILE_02 = 'LINK-05-04-22.xlsx'

df = pd.read_excel(FILE_02, index_col=None)

logger.info('Leido Archivo %s', FILE_02)
logger.info('Numero de registros en Archivo %s', len(df['key']))

2022-04-05 17:32:31,617 - UOCPRA01 - INFO - Leido Archivo LINK-05-04-22.xlsx
2022-04-05 17:32:31,617 - UOCPRA01 - INFO - Numero de registros en Archivo 3102


In [38]:
# Función Lectura Información Webs secundarias de Masters

def WebSecundaria(Link):
    """
    Funcion:
        WebSecundaria(link, indice)
        Version 2.0
    Entrada:
        link -> Link a analizar
     
    Return:
        Lista valores obtenidos
        - Titulo -> Titulo Master
        - Entidad -> Nombre entidad que hace master
        - urlImagen -> Link Imagen
        - nombreImagen -> Nombre Imagen
        - Telefono -> Telefono contacto
        - Precio -> Precio Master
        - Metodologia -> Metodologia Master
        - Lugar -> Lugar Imparticion
        - Duracion -> Duracion Master
        - Tipologia -> Tipologia Master
        - BolsaEmp -> Bolsa Empleo
        - x3 -> Precio Master
        - x4 -> Texto Descriptivo Master
        - x5 -> Tipologia Master
        - x6 -> Programa Master
    """
    # Abrir Web    
    # Captura de posibles errores Webs
    
    # Delays para evitar problemas
    time.sleep(random.randint(1,2))
    
    try:
        html = urlopen(Link)
    except HTTPError as e:
        # Lanza un aviso con indice de ListaWebs y error
        logger.warning('warning %s', e)
        return
    except URLError as e:
        # Lanza aviso si no encuentra la Web
        logger.warning('%s ',e)               
        return

    # Generar objeto BeatifulSoup
    bs = BeautifulSoup(html, 'html.parser')
                       
    # Titulo Master
    for t1 in bs.select(".title-box__name"):
        #print(t1.text)
        x1 = t1.text
        Titulo = x1.strip()
                       
    # Nombre entidad que hace el Master
    for t1 in bs.select(".course-box__link"):
        #print(t1.text)
        Entidad = t1.text.strip()
        
    # Imagen en pagina Master
    a=bs.select(".boxes-untrack__logo > img:nth-child(1)")
    # Si hay imagen
    if a:
        # Link Imagen
        urlImagen=a[0]['src'].strip()
        # Extraer nombre Archivo Imagen
        nombreImagen= ExtraeName(urlImagen)
    else:
        urlImagen=""
        nombreImagen="Sin Imagen"

    # Telefono
    a=bs.select(".app_contactCenterSectionDesktop:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > a:nth-child(1) > span:nth-child(1)")
    if a:
        # Extracción Telefono
        Telefono=a[0].text
        # - Elimina caracteres especiales
        Telefono=re.sub(r"[^a-zA-Z0-9]","",Telefono)
        # - Elimina espacios blanco principio y final
        Telefono=Telefono.strip()
    else:
        Telefono="Sin Datos"
        
     # Precio
    a=bs.select(".app_course_price_box > div:nth-child(2) > div:nth-child(1) > span:nth-child(1)")
    if a:
        # Extracción Precio
        Precio=a[0].text
        # En casos de precios formato "XXXX-YYYY" nos quedamos con XXXX
        Precio=Precio.split("-")[0]
        # - Elimina caracteres especiales
        Precio=re.sub(r"[^a-zA-Z0-9]","",Precio)
        # - Elimina espacios blanco principio y final
        Precio=Precio.strip()
        # - Si todos los caracteres son numericoa convierte a float
        if Precio.isnumeric():
            # Convertir en float
            Precio=float(Precio)
        
    else:
        Precio="Precio a consultar"
    
    # Metodologia
    a=bs.select("ul.course-detail__list:nth-child(1) > li:nth-child(2) > p:nth-child(2) > span:nth-child(1)")
    if a:
        # Metodologia
        Metodologia=a[0].text.strip()
    else:
        Metodologia="Sin Datos"
    
    # Lugar impartición Master
    a=bs.select("ul.course-detail__list:nth-child(1) > li:nth-child(3) > p:nth-child(2) > span:nth-child(1)")
    if a:
        # Lugar
        Lugar=a[0].text.strip()
    else:
        Lugar="Sin Datos"
    
    # TipoCurso
    a=bs.select("ul.course-detail__list:nth-child(1) > li:nth-child(4) > p:nth-child(2) > span:nth-child(1)")
    if a:
        # TipoCurso
        TipoCurso=a[0].text.strip()
    else:
        TipoCurso="Sin Datos"
       
    # Tipologia
    a=bs.select("ul.course-detail__list:nth-child(1) > li:nth-child(1) > p:nth-child(2) > span:nth-child(1)")
    if a:
        # Tipologia
        Tipologia=a[0].text.strip()
    else:
        Tipologia="Sin Datos" 

     # Bolsa Empleo
    a=bs.select("ul.course-detail__list:nth-child(2) > li:nth-child(4) > p:nth-child(2) > span:nth-child(1)")
    if a:
        # Bolsa Empleo (Si/No)
        BolsaEmpl=a[0].text.strip()
    else:
        BolsaEmpl="Sin Datos" 
                       
    # Precio del Master
    # - Si no tiene valor asigna precio Sin Datos
    pa = bs.select("div.price-box:nth-child(1) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1)")
    if not pa:
        x3= "SinDatos"
    # - En el resto de casos
    for t1 in pa:
        # - Obtención del valor
        x31 = t1.text
        # En casos de precios formato "XXXX-YYYY" nos quedamos con XXXX
        x31=x31.split("-")[0]        
        # - Elimina caracteres especiales
        x3=re.sub(r"[^a-zA-Z0-9]","",x31)
        # - Elimina espacios blanco principio y final
        x3=x3.strip()
        # - Si todos los caracteres son numericoa convierte a float
        if x3.isnumeric():
            # Convertir en float
            x3=float(x3)
            # Definirlo como precio ( La información esta en dos sitios )
            Precio=x3
        
    # Texto Descriptivo
    # - Si no tiene valor asigna cadena vacia
    if not bs.select(".course-box__text"):
        x4="SinDatos"
    # - En el resto de casos
    for t1 in bs.select(".course-box__text"):
        # - Obtención del valor
        x41 = t1.text
        # - Eliminamos acentos. Basado en:
# https://es.stackoverflow.com/questions/135707/c%C3%B3mo-puedo-reemplazar-las-letras-con-tildes-por-las-mismas-sin-tilde-pero-no-l
        # -> NFD y eliminar diacríticos
        x4 = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize("NFD", x41), 0, re.I)
        # -> NFC
        x42 = normalize( 'NFC', x4)
        
        # - Eliminamos caracteres especiales
        x4=re.sub(r"[^a-zA-Z0-9]"," ",x42)
        # - Elimina espacios blanco principio y final
        x4=x4.strip()
        
        
    # Tipologia del Master
    # - Si no tiene valor asigna cadena vacia
    if not bs.select(".course-venues__address > span:nth-child(1)"):
        x5=""
    # - En el resto de los casos
    for t1 in bs.select(".course-venues__address > span:nth-child(1)"):
        # Si no hay valores
        if not t1:
            x5="Sin Datos"
        #print(t1.text)
        x5 = t1.text.strip()

    # Programa Master
    texto=[]
    for table in bs.select(".lessons-box.app_lessons-box"):
        for i, tr in enumerate(table.findAll('strong')):
            texto.append(tr.text)
            for td in tr.findAll('li'):
                texto.append(td.text)
    x6 = texto
                       
    # Construcción Lista resultado 
#        - Titulo -> Titulo Master -- 'MASTER'                   
#        - Entidad -> Nombre entidad que hace master -- 'ENTIDAD'
#        - Precio -> Precio Master -- 'PRECIO'
#        - TipoCurso -> Tipo Curso -- 'TIPO_CURSO'
#        - Tipologia -> Tipologia Master -- 'TIPOLOGIA'
#        - Metodologia -> Metodologia Master -- 'METODOLOGIA'
#        - Lugar -> Lugar Imparticion -- LUGAR
#        - BolsaEmp -> Bolsa Empleo -- 'BOLSAEMP'
#        - Telefono -> Telefono contacto --  'TELEFONO'
#        - x4 -> Texto Descriptivo Master
#        - x6 -> Programa Master  
#        - urlImagen -> Link Imagen
#        - nombreImagen -> Nombre Imagen
#        'WEB'

#        - x3 -> Precio Master (NO LO USAREMOS)
#        - x5 -> Tipologia Master (NO LO USAREMOS)
  
    
    logger.info('Procesada %s', x1)                  
    return (Titulo,
            Entidad,
            Precio,
            Tipologia,
            Metodologia,           
            Lugar,          
            BolsaEmpl,
            Telefono,
            x4,
            x6,
            urlImagen,
            nombreImagen,           
            Link)

# 03.- ANALISIS

Tenemos dos opciones:
    
    03.01- SECUENCIAL
    03.02.-MULTIPROCESO

In [ ]:
# 03.01.-PROCESO SECUENCIAL de generación Datos Master

# Datos --> DataFrame df
#  - Columnas:
#       'key'-> Indice
#       'Pagina_Origen' -> Web Origen Busqueda
#     , 'Titulo' -> Nombre Master
#     , 'Pag_Referencia' --> Web de detalle Master
#     

# Construcción Lista Webs a analizar
listadoWebs = df['Pag_Referencia'].tolist()
# Tamaño Datos
#tamano = len(listadoWebs)
tamano = 5

# Control Tiempo
start = time.time()

logger.info('********************************')
logger.info('*   INICIO PROCESO SECUENCIAL  *')
logger.info('********************************')

# Para cada valor del indice
for indice in range(tamano):
    # Paro para evitar bloqueo 
    sleep(1)
    # Construcción Diccionario
    #s = WebSecundaria(listadoWebs[indice], indice)
    resultado_seq = WebSecundaria(listadoWebs[indice])
    #logger.info('%s: Informacion de %s', indice, s[indice][2])

logger.info('********************************')
logger.info('*     FIN PROCESO SECUENCIAL   *')
logger.info('********************************')

end = time.time()
duracion = (end-start)
logger.info('El proceso secuencial duro %s segundos', duracion)

In [28]:
# 03.02.-PROCESO USANDO MULTIPROCESO de generación Datos Master

from multiprocessing import Pool

In [29]:
# Lista de web a analizar

# Construcción Lista Webs a analizar
listadoWebs = df['Pag_Referencia'].tolist()


# Para hacer pruebas limitando el alcance listadoWebs[:limite]
url_list=listadoWebs

In [30]:
url_list

['https://www.emagister.com/master-mba-direccion-centros-hidrotermales-cursos-3572081.htm',
 'https://www.emagister.com/master-nutricion-salud-cursos-3693630.htm',
 'https://www.emagister.com/master-biotecnologia-salud-madrid-cursos-3170196.htm',
 'https://www.emagister.com/master-universitario-nutricion-salud-cursos-3006594.htm',
 'https://www.emagister.com/gestion-sanitaria-para-profesionales-salud-cursos-2801676.htm',
 'https://www.emagister.com/master-salud-mental-cursos-3466528.htm',
 'https://www.emagister.com/master-trabajo-social-servicios-salud-master-salud-mental-cursos-3658125.htm',
 'https://www.emagister.com/master-nutricion-salud-distancia-cursos-3693631.htm',
 'https://www.emagister.com/master-salud-mental-anciano-master-enfermedad-del-alzheimer-practicas-cursos-3544150.htm',
 'https://www.emagister.com/master-nutricion-salud-coaching-gestion-emocional-mindfulness-distancia-cursos-3693680.htm',
 'https://www.emagister.com/master-direccion-centros-medicos-clinicas-hospita

In [39]:
# https://medium.com/@kunal.rustagi/boost-your-web-crawler-using-multiple-processes-in-python-3cc3ff519226

start = time.time()

logger.info('********************************')
logger.info('*   INICIO MULTIPROCESO        *')
logger.info('********************************')

# Numero procesos simultaneos
num_proc = 10
logger.info('Procesos simultaneos: %s',str(num_proc))


p = Pool(num_proc)
resultados_mul = p.map(WebSecundaria, url_list)
p.terminate()
p.join()



logger.info('********************************')
logger.info('*     FIN MULTIPROCESO         *')
logger.info('********************************')

end = time.time()
duracion = (end-start)
logger.info('La secuencia Multiproceso duro %s segundos', duracion)

2022-04-05 18:05:41,715 - UOCPRA01 - INFO - ********************************
2022-04-05 18:05:41,717 - UOCPRA01 - INFO - *   INICIO MULTIPROCESO        *
2022-04-05 18:05:41,718 - UOCPRA01 - INFO - ********************************
2022-04-05 18:05:44,064 - UOCPRA01 - INFO - Procesada  POSTGRADO EN REHABILITACIÓN ACUÁTICA 
2022-04-05 18:05:44,185 - UOCPRA01 - INFO - Procesada  Postgrado en eSports Digital Business 
2022-04-05 18:05:44,207 - UOCPRA01 - INFO - Procesada  Cursos Máster Propio en Contabilidad 
2022-04-05 18:05:44,312 - UOCPRA01 - INFO - Procesada  MÁSTER EN ACUPUNTURA - ONLINE 
2022-04-05 18:05:44,318 - UOCPRA01 - INFO - Procesada  MASTER EN ORGANIZACIÓN Y CONTROL DE LA FABRICACIÓN DE PRODUCTOS FARMACÉUTICOS 
2022-04-05 18:05:44,333 - UOCPRA01 - INFO - Procesada  Máster MBA en dirección de centros hidrotermales 
2022-04-05 18:05:44,336 - UOCPRA01 - INFO - Procesada  MÁSTER MBA EXPERTO EN DIRECCIÓN Y GESTIÓN DE PYME'S - CERTIFICACIÓN UNIVERSITARIA INTERNACIONAL + 60 ECTS - 


2022-04-05 18:06:08,230 - UOCPRA01 - INFO - Procesada  Doctorado en nutrición 
2022-04-05 18:06:08,409 - UOCPRA01 - INFO - Procesada  Máster Universitario en Dirección Sanitaria y Gestión Clínica 
2022-04-05 18:06:08,564 - UOCPRA01 - INFO - Procesada  Máster en Comercio Internacional (Presencial) 
2022-04-05 18:06:08,652 - UOCPRA01 - INFO - Procesada  MÁSTER EN PERITO JUDICIAL EN AUTOMATISMOS INDUSTRIALES 
2022-04-05 18:06:08,673 - UOCPRA01 - INFO - Procesada  Máster Online en Psicología Positiva Aplicada. ¡SOLICITA YA TU BECA! 
2022-04-05 18:06:08,692 - UOCPRA01 - INFO - Procesada  Máster en márketing digital y eCommerce con Titulación 
2022-04-05 18:06:08,735 - UOCPRA01 - INFO - Procesada 
                     Programa Ejecutivo en Digital Customer Experience             
2022-04-05 18:06:08,794 - UOCPRA01 - INFO - Procesada  Máster en Nutrición y Salud - A distancia 
2022-04-05 18:06:08,884 - UOCPRA01 - INFO - Procesada  Máster en Enfermería Veterinaria 
2022-04-05 18:06:10,566 - UO

2022-04-05 18:06:29,103 - UOCPRA01 - INFO - Procesada  Máster en Nutrición y Salud & Máster en Coaching Deportivo 
2022-04-05 18:06:29,399 - UOCPRA01 - INFO - Procesada  Máster en dirección y gestión del talento 
2022-04-05 18:06:29,441 - UOCPRA01 - INFO - Procesada  Posgrado en Coaching, PNL y Gestión de Equipos 
2022-04-05 18:06:29,743 - UOCPRA01 - INFO - Procesada  MÁSTER EN COMERCIO INTERNACIONAL Y EXPORTACIÓN - DIPLOMA AUTENTIFICADO POR NOTARIO EUROPEO - 
2022-04-05 18:06:29,986 - UOCPRA01 - INFO - Procesada  Máster online de Humanización en Salud 
2022-04-05 18:06:30,118 - UOCPRA01 - INFO - Procesada  Master oficial universitario en dirección y gestión sanitaria 
2022-04-05 18:06:30,210 - UOCPRA01 - INFO - Procesada  Máster en psicología infantil y adolescente más máster en coaching y en inteligencia emocional infantil y juvenil 
2022-04-05 18:06:30,316 - UOCPRA01 - INFO - Procesada  Máster en Dirección y Gestión de Proyectos - Project Management 
2022-04-05 18:06:30,513 - UOCPRA

2022-04-05 18:06:53,832 - UOCPRA01 - INFO - Procesada  MÁSTER EN TIPOGRAFÍA Y MAQUETACIÓN 
2022-04-05 18:06:53,986 - UOCPRA01 - INFO - Procesada  Máster en nutrición y dietética 
2022-04-05 18:06:54,050 - UOCPRA01 - INFO - Procesada  Máster en atención de enfermería en urgencias y emergencias 
2022-04-05 18:06:54,120 - UOCPRA01 - INFO - Procesada  Máster experto en psicología y máster en counseling y terapia gestalt 
2022-04-05 18:06:54,560 - UOCPRA01 - INFO - Procesada  Máster Comercio Internacional Y Exportación - con posibilidad de prácticas 
2022-04-05 18:06:54,617 - UOCPRA01 - INFO - Procesada  Master en Gestión de la Seguridad Clínica del Paciente y Calidad en la Atención Sanitaria 
2022-04-05 18:06:54,937 - UOCPRA01 - INFO - Procesada  MBA - Máster en Administración y Dirección de Empresas 
2022-04-05 18:06:55,330 - UOCPRA01 - INFO - Procesada  Máster Universitario online en Gestión de Proyectos - Project Management 
2022-04-05 18:06:55,349 - UOCPRA01 - INFO - Procesada  Máster 

2022-04-05 18:07:14,572 - UOCPRA01 - INFO - Procesada  Máster Universitario en Emprendimiento de Negocios de Base Tecnológica 
2022-04-05 18:07:15,015 - UOCPRA01 - INFO - Procesada  Máster experto en logopedia, trastornos del lenguaje y del habla 
2022-04-05 18:07:15,139 - UOCPRA01 - INFO - Procesada  MÁSTER AUXILIAR DE ODONTOLOGÍA 
2022-04-05 18:07:15,193 - UOCPRA01 - INFO - Procesada  Máster en control sanitario y bioseguridad en aves y granjas 
2022-04-05 18:07:15,247 - UOCPRA01 - INFO - Procesada  Programa superior universitario en dirección de recursos humanos 
2022-04-05 18:07:16,010 - UOCPRA01 - INFO - Procesada  MÁSTER EN DISEÑO NAVAL 
2022-04-05 18:07:17,034 - UOCPRA01 - INFO - Procesada  Master en Instalaciones Térmicas: Eficiencia Energética 
2022-04-05 18:07:17,105 - UOCPRA01 - INFO - Procesada  Máster en Emprendimiento e Innovación 
2022-04-05 18:07:17,182 - UOCPRA01 - INFO - Procesada  Máster de formación permanente en urgencias y emergencias en enfermería 
2022-04-05 18:

...

2022-04-05 18:20:39,041 - UOCPRA01 - INFO - Procesada  RUSO BÁSICO EN ACTIVIDADES DE VENTA Y TURISMO 
2022-04-05 18:20:39,046 - UOCPRA01 - INFO - Procesada  Programa superior Organización de Eventos, Protocolo y Turismo de Negocio 
2022-04-05 18:20:39,151 - UOCPRA01 - INFO - Procesada  Curso Intensivo de Italiano básico - ONLINE 
2022-04-05 18:20:39,164 - UOCPRA01 - INFO - Procesada  MÁSTER FITNESS Y CARDIO BOX + MÁSTER COACHING DEPORTIVO 
2022-04-05 18:20:39,170 - UOCPRA01 - INFO - Procesada  MÁSTER EN ATENCIÓN PEDIÁTRICA 
2022-04-05 18:20:39,232 - UOCPRA01 - INFO - Procesada  Máster en personal trainer y coaching deportivo 
2022-04-05 18:20:39,258 - UOCPRA01 - INFO - Procesada  Pregrado en Lenguas y Cultura 
2022-04-05 18:20:41,145 - UOCPRA01 - INFO - Procesada  Máster en logística y transporte 
2022-04-05 18:20:41,352 - UOCPRA01 - INFO - Procesada  MBA y Especialista en dirección y gestión de entidades deportivas 
2022-04-05 18:20:41,504 - UOCPRA01 - INFO - Procesada 
              

2022-04-05 18:20:59,838 - UOCPRA01 - INFO - Procesada  Máster en Coaching Deportivo 
2022-04-05 18:20:59,959 - UOCPRA01 - INFO - Procesada  Master Logística y Dirección de Almacén + Titulación Universitaria 
2022-04-05 18:21:00,054 - UOCPRA01 - INFO - Procesada  Máster Universitario en Gestión Internacional del Turismo 
2022-04-05 18:21:00,655 - UOCPRA01 - INFO - Procesada  Italiano A2 
2022-04-05 18:21:00,959 - UOCPRA01 - INFO - Procesada  Diplomado en Seguridad Nacional en México: Amenazas Asimétricas, versión en línea 
2022-04-05 18:21:01,078 - UOCPRA01 - INFO - Procesada  Master en Traducción Especializada (Mención Francés) + 60 Créditos ECTS 
2022-04-05 18:21:01,228 - UOCPRA01 - INFO - Procesada  Produccion y venta de servicios turisticos en agencia de viajes (CURSO ONLINE CON PRÁCTICAS PRESENCIALES) 
2022-04-05 18:21:01,990 - UOCPRA01 - INFO - Procesada  Máster universitario en dirección en la gestión pública 
2022-04-05 18:21:02,708 - UOCPRA01 - INFO - Procesada  Doctorado en Tr

2022-04-05 18:21:20,782 - UOCPRA01 - INFO - Procesada  Máster Universitario en Gestión de Contratos y Programas del Sector Público, Con Especial Aplicación Al ámbito de la Defensa 
2022-04-05 18:21:20,847 - UOCPRA01 - INFO - Procesada  Master MBA en dirección de empresas de restauración y gastronomía 
2022-04-05 18:21:20,922 - UOCPRA01 - INFO - Procesada  Máster en scouting en fútbol (análisis, informes y decisiones) más máster en coaching deportivo 
2022-04-05 18:21:21,282 - UOCPRA01 - INFO - Procesada  Experto universitario en Planificación de Programas Fitness Acuático 
2022-04-05 18:21:21,651 - UOCPRA01 - INFO - Procesada  Máster en Contratación y Técnicas de Negociación en el Transporte por Carretera - con posibilidad de prácticas 
2022-04-05 18:21:22,164 - UOCPRA01 - INFO - Procesada  Clases particulares de coreano 
2022-04-05 18:21:22,241 - UOCPRA01 - INFO - Procesada  Traducción profesional francés-español (online) 
2022-04-05 18:21:22,421 - UOCPRA01 - INFO - Procesada  Master 

2022-04-05 18:21:41,806 - UOCPRA01 - INFO - Procesada  Máster Experto En Transporte Sanitario 
2022-04-05 18:21:41,923 - UOCPRA01 - INFO - Procesada  UF2373 Usuario Independiente de LSE, Nivel de Competencia Avanzado, B2.1 
2022-04-05 18:21:42,105 - UOCPRA01 - INFO - Procesada  Máster en Enseñanza del Español como Lengua Extranjera 
2022-04-05 18:21:42,456 - UOCPRA01 - INFO - Procesada  Máster en desarrollo local 
2022-04-05 18:21:42,853 - UOCPRA01 - INFO - Procesada  Maestría en Administración Pública 
2022-04-05 18:21:43,123 - UOCPRA01 - INFO - Procesada  Mantenimiento y conservación de barcos 
2022-04-05 18:21:43,573 - UOCPRA01 - INFO - Procesada 
                     Máster Universitario en Enseñanza del Español como Lengua Extranjera             
2022-04-05 18:21:44,066 - UOCPRA01 - INFO - Procesada  Técnico experto en operaciones aeroportuarias.. 
2022-04-05 18:21:44,688 - UOCPRA01 - INFO - Procesada  Curso Intensivo Inglés B2 + Especialización en Enseñanza del Inglés para Maestr

2022-04-05 18:22:09,841 - UOCPRA01 - INFO - Procesada  Diploma de extensión universitaria de director de seguridad 
2022-04-05 18:22:12,471 - UOCPRA01 - INFO - Procesada  Procesos y estrategias de aprendizaje de ELE 
2022-04-05 18:22:13,137 - UOCPRA01 - INFO - Procesada  Máster Internacional Online en Paz Seguridad y Defensa 
2022-04-05 18:22:14,794 - UOCPRA01 - INFO - Procesada  Clases particulares de japonés para principiantes 
2022-04-05 18:22:15,574 - UOCPRA01 - INFO - Procesada  Máster en derecho administrativo 
2022-04-05 18:22:17,803 - UOCPRA01 - INFO - Procesada  MÁSTER EN SEGURIDAD EN PARQUES INFANTILES 
2022-04-05 18:22:17,925 - UOCPRA01 - INFO - ********************************
2022-04-05 18:22:17,926 - UOCPRA01 - INFO - *     FIN MULTIPROCESO         *
2022-04-05 18:22:17,927 - UOCPRA01 - INFO - ********************************
2022-04-05 18:22:17,927 - UOCPRA01 - INFO - La secuencia Multiproceso duro 996.2126264572144 segundos


In [40]:
# 03.03.- Creación Dataframe resultado

DfResult = pd.DataFrame(resultados_mul, columns = ['MASTER',
                                                   'ENTIDAD',
                                                   'PRECIO',
                                                   'TIPOLOGIA',
                                                   'METODOLOGIA',
                                                   'DURACION',
                                                   'BOLSAEMP',
                                                   'TELEFONO',
                                                   'X4',
                                                   'X6',
                                                   'URLIMAGEN',
                                                   'NOMBREIMAGEN',
                                                   'WEB'])

In [41]:
DfResult

,MASTER,ENTIDAD,PRECIO,TIPOLOGIA,METODOLOGIA,DURACION,BOLSAEMP,TELEFONO,X4,X6,URLIMAGEN,NOMBREIMAGEN,WEB
0,Máster MBA en dirección de centros hidrotermales,SELECT BUSINESS SCHOOL,1940.0,Master,Semipresencial,Madrid,Sí,916358357,Plantea tus dudas y otros usuarios podran resp...,"[Contenido formativo, PARTE 1. GESTIÓN LABORAL...",https://www.emagister.com/assets/es/logos/cent...,['l.png'],https://www.emagister.com/master-mba-direccion...
1,Máster en Nutrición y Salud,ISEB - Instituto Superior Europeo de Barcelona,1250.0,Master,Online,12 Meses,Sin Datos,932824831,Plantea tus dudas y otros usuarios podran resp...,[],https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/master-nutricion-sal...
2,Máster en Biotecnología de la Salud - Madrid,CESIF Formación,Precio a consultar,Master,Madrid,1840h,Sin Datos,912669065,Plantea tus dudas y otros usuarios podran resp...,[SECTOR INDUSTRIAL EN BIOTECNOLOGIA DE LA SALU...,https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/master-biotecnologia...
3,Máster Universitario en Nutrición y Salud,Universidad Internacional de Valencia (VIU),Precio a consultar,Master oficial,Online,Sin Datos,Sin Datos,962642879,El principal objetivo del Master Oficial Onlin...,[],https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/master-universitario...
4,Máster Profesional en Gestión Sanitaria,Cef - Centro de Estudios Financieros,5400.0,Master oficial,Online,Sin Datos,Sin Datos,912669065,Si trabajas en el sector sanitario y quieres a...,[Programa \n\n],https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/gestion-sanitaria-pa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3097,MÁSTER EN SEGURIDAD EN PARQUES INFANTILES,ESNECA BUSINESS SCHOOL,380.0,Master,A distancia,600h,Sin Datos,973975855,Plantea tus dudas y otros usuarios podran resp...,[],https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/master-seguridad-par...
3098,Máster Universitario en Seguridad y Defensa,A+ CAPEC,25700.0,Master,Sydney (Australia),Sin Datos,Sin Datos,912669065,Universidad Nebrija,"[Asignaturas Troncales, Asignaturas Troncales,...",https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/master-of-internatio...
3099,Master Oficial Universitario en Policía Cientí...,UB - Universitat de Barcelona,3495.0,Master,Barcelona,Sin Datos,Sin Datos,932933037,ENALDE BUSINESS SCHOOL,[],https://www.emagister.com/assets/es/logos/cent...,['l.png'],https://www.emagister.com/master-oficial-polic...
3100,Máster en derecho administrativo,Acath Formació,2001.0,Postgrado,Santa coloma de farners,Sin Datos,Sin Datos,932933037,ESCUELA DE POSTGRADO DE CIENCIAS DEL DERECHO,[],https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/postgrado-adiestrami...


In [42]:
DfResult.columns

Index(['MASTER', 'ENTIDAD', 'PRECIO', 'TIPOLOGIA', 'METODOLOGIA', 'DURACION',
       'BOLSAEMP', 'TELEFONO', 'X4', 'X6', 'URLIMAGEN', 'NOMBREIMAGEN', 'WEB'],
      dtype='object')

In [43]:
# 03.04.01.- Generación Archivo parcial de resultados (.xlsx)

# Generación Nombres Archivos a generar
from datetime import datetime

nombre=datetime.today().strftime('%d-%m-%y')

FILE_02 = "RESULT-"+nombre+".xlsx"

# Directorio donde se generara salida
path='RESULTADOS'
file1=GenDirecFile(path, FILE_02)

# Escritura
DfResult.to_excel(file1)
logger.info('Generado Archivo %s', FILE_02)

2022-04-05 18:22:38,749 - UOCPRA01 - INFO - Generado Archivo RESULT-05-04-22.xlsx


In [44]:
# 03.04.02.- Generación Archivo parcial de resultados (CSV)

# Generación Nombres Archivos a generar
from datetime import datetime

nombre=datetime.today().strftime('%d-%m-%y')

FILE_03 = "RESULT-"+nombre+".csv"

# Directorio donde se generara salida
path='RESULTADOS'
filecsv=GenDirecFile(path, FILE_03)

DfResult.to_csv(filecsv, index = False)
logger.info('Generado Archivo %s', FILE_03)

2022-04-05 18:23:32,161 - UOCPRA01 - INFO - Generado Archivo RESULT-05-04-22.csv


In [22]:
# Extracción Imagenes de las WEBs

# Construccion diccionario imagenes de las entidades

# Filtrado Entidades con Logo
#DfResultLOGO=DfResult[DfResult['URLIMAGEN']!='']

# Lista entidades unicas
listaOrganizaciones = DfResult['ENTIDAD'].unique().tolist()

# Construccion diccionario imagenes de las entidades
DiccioImagenes = {}
# Para cada entidad
for s in listaOrganizaciones:
    # Hacer entrada por entidad con lista de imagenes de la entidad
    DiccioImagenes[s] = DfResult[DfResult['ENTIDAD'] == s]['URLIMAGEN'].unique().tolist()
    
logger.info('Generado Diccionario de tamaño %s', len(DiccioImagenes))   



2022-04-02 21:10:11,963 - UOCPRA01 - INFO - Generado Diccionario de tamaño 195


In [23]:
# Prueba

DiccioImagenes['Instituto Español de Formación Social']

['https://www.emagister.com/assets/es/logos/centro/id/192475/size/l.png']

In [24]:
# Directorio Imagenes
Path = 'IMAGENES'

logger.info('Inicio Descarga Imagenes')

# Itera en el diccionario Previo
for k, v in DiccioImagenes.items():
    # Descarga Archivo en Path
    # v --> Lista de Links de imagenes
    # k --> Institucion
    # Archivo resultante con nombre
    logger.info('Descargando Logo de %s', k)
    # Pasar lista a String
    v = v[0]
    logger.info('Link %s', v)
    # - Eliminamos caracteres especiales
    k=re.sub(r"[^a-zA-Z0-9]"," ",k)
    # - Elimina espacios blanco principio y final
    k=k.strip()
    DescargaImagenes(Path, v, k)

logger.info('Fin Descarga Imagenes')

2022-04-02 21:10:11,978 - UOCPRA01 - INFO - Inicio Descarga Imagenes
2022-04-02 21:10:11,980 - UOCPRA01 - INFO - Descargando Logo de SELECT BUSINESS SCHOOL
2022-04-02 21:10:11,981 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/192308/size/l.png
2022-04-02 21:10:12,988 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:10:13,210 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/192308/size/l.png HTTP/1.1" 200 13231
2022-04-02 21:10:13,215 - UOCPRA01 - INFO - Descargando Logo de CESIF Formación
2022-04-02 21:10:13,216 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/56155/size/l.jpg
2022-04-02 21:10:15,223 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:10:15,486 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/56155/size/l.jp

2022-04-02 21:10:42,470 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/193808/size/l.jpg HTTP/1.1" 200 6330
2022-04-02 21:10:42,474 - UOCPRA01 - INFO - Descargando Logo de ESCUELA INTERNACIONAL DE FARMACIA PASTEUR
2022-04-02 21:10:42,476 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1140014/size/l.jpg
2022-04-02 21:10:43,479 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:10:43,686 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1140014/size/l.jpg HTTP/1.1" 200 4908
2022-04-02 21:10:43,690 - UOCPRA01 - INFO - Descargando Logo de Editorial Médica Panamericana
2022-04-02 21:10:43,692 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/94945/size/l.jpg
2022-04-02 21:10:44,697 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21

2022-04-02 21:11:10,787 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1721/size/l.jpg HTTP/1.1" 200 19366
2022-04-02 21:11:10,792 - UOCPRA01 - INFO - Descargando Logo de Universidad a Distancia de Madrid - Masters
2022-04-02 21:11:10,793 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/202616/size/l.jpg
2022-04-02 21:11:12,800 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:11:13,056 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/202616/size/l.jpg HTTP/1.1" 200 14787
2022-04-02 21:11:13,058 - UOCPRA01 - INFO - Descargando Logo de VEIGLER BUSINESS SCHOOL
2022-04-02 21:11:13,059 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1153673/size/l.jpg
2022-04-02 21:11:15,065 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:11:

2022-04-02 21:11:43,226 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1168562/size/l.jpg HTTP/1.1" 200 21678
2022-04-02 21:11:43,232 - UOCPRA01 - INFO - Descargando Logo de SEFHOR
2022-04-02 21:11:43,234 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1153674/size/l.jpg
2022-04-02 21:11:44,240 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:11:44,509 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1153674/size/l.jpg HTTP/1.1" 200 30966
2022-04-02 21:11:44,511 - UOCPRA01 - INFO - Descargando Logo de Edil formación
2022-04-02 21:11:44,512 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1123086/size/l.jpg
2022-04-02 21:11:46,518 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:11:46,737 - urllib3.connectionpool - DEBUG -

2022-04-02 21:12:13,637 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/55701/size/l.jpg HTTP/1.1" 200 8541
2022-04-02 21:12:13,642 - UOCPRA01 - INFO - Descargando Logo de Escuela de Postgrado de Veterinaria
2022-04-02 21:12:13,644 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1152427/size/l.jpg
2022-04-02 21:12:14,650 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:12:14,863 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1152427/size/l.jpg HTTP/1.1" 200 6882
2022-04-02 21:12:14,868 - UOCPRA01 - INFO - Descargando Logo de Instituto Europeo de Química, Física y Biología - IEQFB
2022-04-02 21:12:14,869 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1155074/size/l.jpg
2022-04-02 21:12:15,876 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.

...

2022-04-02 21:13:40,405 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:13:40,617 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1160425/size/l.jpg HTTP/1.1" 200 8531
2022-04-02 21:13:40,621 - UOCPRA01 - INFO - Descargando Logo de Spain Business School - Formación
2022-04-02 21:13:40,623 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/139196/size/l.jpg
2022-04-02 21:13:42,630 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:13:42,894 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/139196/size/l.jpg HTTP/1.1" 200 31740
2022-04-02 21:13:42,899 - UOCPRA01 - INFO - Descargando Logo de ESCUELA DE POSTGRADO DE INGENIERIA Y ARQUITECTURA
2022-04-02 21:13:42,901 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1155659/size/l.jpg

2022-04-02 21:14:09,697 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:14:09,962 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1137833/size/l.jpg HTTP/1.1" 200 46066
2022-04-02 21:14:10,013 - UOCPRA01 - INFO - Descargando Logo de Instituto Idem
2022-04-02 21:14:10,014 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/60835/size/l.jpg
2022-04-02 21:14:11,019 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:14:11,235 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/60835/size/l.jpg HTTP/1.1" 200 8209
2022-04-02 21:14:11,238 - UOCPRA01 - INFO - Descargando Logo de SAN ANASTASIO INTERNATIONAL SCHOOL
2022-04-02 21:14:11,239 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1152478/size/l.jpg
2022-04-02 21:14:12,244 - urllib3.c

2022-04-02 21:14:39,099 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1168850/size/l.jpg
2022-04-02 21:14:40,105 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:14:40,321 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1168850/size/l.jpg HTTP/1.1" 200 6620
2022-04-02 21:14:40,326 - UOCPRA01 - INFO - Descargando Logo de THREE POINTS
2022-04-02 21:14:40,328 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1137887/size/l.jpg
2022-04-02 21:14:42,334 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:14:42,551 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1137887/size/l.jpg HTTP/1.1" 200 9297
2022-04-02 21:14:42,556 - UOCPRA01 - INFO - Descargando Logo de CEAM
2022-04-02 21:14:42,558 - UOCPRA01 - INFO - Link https://www.ema

2022-04-02 21:15:07,400 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/59251/size/l.jpg
2022-04-02 21:15:09,407 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:15:09,669 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/59251/size/l.jpg HTTP/1.1" 200 10222
2022-04-02 21:15:09,674 - UOCPRA01 - INFO - Descargando Logo de AITEX Instituto Tecnológico Textil
2022-04-02 21:15:09,675 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/81552/size/l.jpg
2022-04-02 21:15:10,681 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:15:10,895 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/81552/size/l.jpg HTTP/1.1" 200 5796
2022-04-02 21:15:10,900 - UOCPRA01 - INFO - Descargando Logo de UNED
2022-04-02 21:15:10,902 - UOCPRA01 - INFO - Link 

2022-04-02 21:15:39,747 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1154628/size/l.jpg
2022-04-02 21:15:40,753 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:15:40,969 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1154628/size/l.jpg HTTP/1.1" 200 7376
2022-04-02 21:15:40,974 - UOCPRA01 - INFO - Descargando Logo de UDL - Universitat de Lleida. Màsters Oficials
2022-04-02 21:15:40,975 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/56021/size/l.jpg
2022-04-02 21:15:41,981 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:15:42,189 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/56021/size/l.jpg HTTP/1.1" 200 4729
2022-04-02 21:15:42,194 - UOCPRA01 - INFO - Descargando Logo de Net-Learning - Entornos virtuales de apr